In [5]:
import sys
from helper import *
from datetime import datetime, timedelta
from indicators import SslChannel, ExponentialMovingAverage
from strategies import SslChannelEmaStrategy
import mplfinance as mpf

In [2]:
ssl_channel_ema_strategy = SslChannelEmaStrategy()

In [6]:
candlestick_5m, candlestick_1h = ssl_channel_ema_strategy.calculate_current_trend()

type(result_points[idx]) <class 'dict'>


In [ ]:
ssl_channel = SslChannel()
ema = ExponentialMovingAverage()

In [ ]:
candlestick_df_ssl = ssl_channel.apply_indicator(candlestick_df)
candlestick_df_ssl_ema = ema.apply_indicator(candlestick_df_ssl)

In [ ]:
candlestick_df_ssl_ema.tail()

In [ ]:
# Plots
apds = [
    mpf.make_addplot(candlestick_df_ssl_ema[['ssl_channel_ssl_down']], color='violet'),
    mpf.make_addplot(candlestick_df_ssl_ema[['ssl_channel_ssl_up']], color='blue'),
    mpf.make_addplot(candlestick_df_ssl_ema[['ema_10']], color='orange'),
    mpf.make_addplot(candlestick_df_ssl_ema[['ssl_channel_trend']])
]


mpf.plot(candlestick_df_ssl_ema, type='candle', style='charles', addplot=apds)